## R1蒸馏

### 加载数据集

AutoDL开启学术加速

In [1]:
# import subprocess
# import os

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value

加载数据集

In [2]:
from datasets import load_dataset
data = load_dataset('openai/gsm8k', 'main', split='train')

/opt/anaconda3/envs/myLM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data

Dataset({
    features: ['question', 'answer'],
    num_rows: 7473
})

### 生成蒸馏数据

库 和 配置参数

In [ ]:
from openai import OpenAI
import os
import threading
import time
import json

API_KEY = "" # todo: 填入你的 aliyun API Key
BASE_URL = 'https://dashscope.aliyuncs.com/compatible-mode/v1'
MODEL_NAME = 'deepseek-r1'

# 定义提示模板，用于指导模型回答问题的风格和要求
PROMPT = '''
# Role
I am an elementary school math tutor. Currently, I will give a clear and easy-to-understand explanation for a specific elementary school math word problem.
Note that the response should be in the form of spoken textual expression, and avoid including the formats such as pictures, hyperlinks, etc.

# Precautions
1. Fully consider the cognitive level and knowledge reserve of primary school students.
2. If the problem is complex, break it down into simple steps. Avoid using professional terms and explain it in plain language.
3. Make more use of real-life examples and intuitive teaching aids to assist in teaching.

# Teaching Style
Adopt a step-by-step teaching method, combined with interactive Q&A sessions and classroom exercises; encourage students through positive feedback to strengthen their understanding of mathematical concepts.

# Math Word Problem for Lesson Preparation
{question} 
'''

# 定义线程数量
THREAD = 30
# 定义要处理的样本数量
SAMPLES = 1000

R1Generator类，使用多线程，用于管理模型生成回答的过程

In [5]:
class R1Generator:
    def __init__(self,threads,dataset,samples):
        # 创建OpenAI客户端，用于与API进行交互
        self.client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
        # 当前处理的样本索引
        self.idx = 0
        # 线程数量
        self.threads = threads
        # 数据集
        self.dataset = dataset
        # 样本数量
        self.samples = samples
        # 线程锁，用于多线程同步
        self.mutex = threading.Lock()

    def generate(self,question):
        # 使用OpenAI API生成回答
        completion=self.client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {'role': 'user', 'content': PROMPT.format(question=question)},
            ]
        )
        print(f"{completion.choices[0].message.reasoning_content}")
        # 返回推理内容和答案
        return completion.choices[0].message.reasoning_content,completion.choices[0].message.content

    def begin(self):
        # 重置索引和进度
        self.idx = 0
        self.progress = 0
        # 初始化结果列表，长度为样本数量，初始值为None
        self.result = [None] * self.samples
        # 初始化线程处理列表
        self.thread_handlers = []
        # 创建并启动线程
        for i in range(self.threads):
            t = threading.Thread(target=self.thread_main)
            t.start()
            self.thread_handlers.append(t)

    def join(self):
        # 等待所有线程完成任务
        while True:
            with self.mutex:
                # 打印当前进度
                print(f'Progress: {self.progress}/{self.samples}', end='\r')
                # 如果所有样本都已处理，则退出循环
                if self.progress >= self.samples:
                    break
            # 暂停1秒
            time.sleep(1)
        # 等待所有线程结束
        for t in self.thread_handlers:
            t.join()
        # 返回处理结果，过滤掉None值
        return [res for res in self.result if res is not None] 

    def thread_main(self):
        # 线程的主要工作函数
        while True:
            with self.mutex:
                # 如果所有样本都已处理，则退出循环
                if self.idx >= self.samples:
                    break
                # 获取当前处理的样本索引
                cur_idx = self.idx
                # 增加索引值
                self.idx += 1
            try:
                # 从数据集中获取问题
                question = self.dataset[cur_idx]['question']
                # 生成回答
                reasoning, answer = self.generate(question)
                # 将问题、推理和答案保存到结果列表中
                self.result[cur_idx] = (question, reasoning, answer)
            except:
                # 如果出现异常，忽略
                pass
            with self.mutex:
                # 增加进度值
                self.progress += 1

主函数

In [6]:
gsm8k=data
r1=R1Generator(threads=THREAD,dataset=gsm8k,samples=SAMPLES)
r1.begin()
result=r1.join()

with open('r1_distill.txt','w') as f:
    for res in result:
        question,reasoning,answer=res
        f.write(json.dumps({'question':question,'reasoning':reasoning,'answer':answer})+'\n')

Okay, let's tackle Betty's money problem step by step. First, I need to figure out how much Betty already has. The wallet costs $100, and she has half of that. Half of 100 is 50, so she starts with $50.

Next, her parents give her $15. Adding that to her savings, she now has 50 plus 15. Let me do the math here: 50 + 15 equals 65. So now she's up to $65.

Then, her grandparents give her twice as much as her parents did. Since her parents gave $15, twice that would be 15 times 2. Let me calculate that: 15 * 2 is 30. So grandparents contribute $30. Adding that to the previous total: 65 + 30 equals 95. Now Betty has $95.

Finally, to find out how much more she needs, subtract what she has from the total cost. The wallet is $100, so 100 minus 95. That leaves her needing $5 more. I should double-check each step to make sure I didn't make any mistakes. Yep, each addition and multiplication step checks out. So the answer is $5.
Okay, let's see. I need to explain how Mike and Johnson share thei